In [1]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchmetrics import Accuracy

import pandas as pd
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
train_data = datasets.CIFAR10(root='./data',
                              train=True,
                              transform=ToTensor(),
                              download=True
                            )

test_data = datasets.CIFAR10(root='./data',
                              train=False,
                              transform=ToTensor(),
                              download=True
                            )

Files already downloaded and verified
Files already downloaded and verified


In [3]:
BATCH_SIZE = 32

train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [4]:
test_img = next(iter(train_dataloader))[0][0].unsqueeze(dim=0)
test_img.shape

torch.Size([1, 3, 32, 32])

In [7]:
# Build the model

class ScratchNetwork(nn.Module):
    def __init__(self, input_channels, hidden_units, output_shape, kernel_size):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(input_channels, hidden_units, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*6*6, output_shape)
        )
        
    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x
    
model = ScratchNetwork(3, 64, 10, 3)
model(test_img)

tensor([[-0.0839, -0.0182, -0.0718,  0.0206, -0.0569,  0.0599, -0.0333, -0.0550,
          0.1052, -0.1093]], grad_fn=<AddmmBackward0>)

In [8]:
from sklearn.metrics import accuracy_score

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

def train_step(model,
               dataloader,
               optimizer,
               loss_fn,
               device
              ):
    
    model.train()
    
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        
        # forward pass
        logits = model(X)
        y_pred = torch.softmax(logits, dim=1).argmax(dim=1)
        
        # calculate loss
        loss = loss_fn(logits, y)
        acc = accuracy_score(y_pred, y)
        train_loss += loss.item()
        train_acc += acc
        
        # zero Grad
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    train_loss = train_loss / len(dataloader) 
    train_acc = train_acc / len(dataloader)
    
    return train_loss, train_acc
        
def test_step(model,
              dataloader,
              optimizer,
              loss_fn,
              device
             ):
    
    model.eval()
    
    test_loss, test_acc = 0, 0
    
    with torch.inference_mode():
        
        for batch, (X, y) in enumerate(dataloader):
            
            logits = model(X)
            y_pred = torch.softmax(logits, dim=1).argmax(dim=1)
            
            loss = loss_fn(logits, y)
            acc = accuracy_score(y_pred, y)
            
            test_loss += loss.item()
            test_acc += acc
            
        test_loss = test_loss / len(dataloader)
        test_acc = test_acc / len(dataloader)
        
        return test_loss, test_acc
    

In [9]:
def train(model,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn,
          device,
          epochs
         ):
    
    for epoch in range(epochs):
        train_loss, train_acc = train_step(model, train_dataloader, optimizer, loss_fn, device)
        test_loss, test_acc = test_step(model, test_dataloader, optimizer, loss_fn, device)
        
        if epoch % 5 == 0:
            print(
                f'epoch {epoch} |'
                f'train loss: {train_loss:.4f} | train acc: {test_acc:.4f} |'
                f'test loss: {test_loss:.4f} | test acc: {test_acc:.4f}'
                 )

In [10]:
train(model, train_dataloader, test_dataloader, optimizer, loss_fn, device, epochs=25)

epoch 0 |train loss: 2.0182 | train acc: 0.3553 |test loss: 1.8085 | test acc: 0.3553
epoch 5 |train loss: 1.2438 | train acc: 0.5468 |test loss: 1.3286 | test acc: 0.5468
epoch 10 |train loss: 1.0502 | train acc: 0.6300 |test loss: 1.0802 | test acc: 0.6300
epoch 15 |train loss: 0.9309 | train acc: 0.6610 |test loss: 0.9882 | test acc: 0.6610
epoch 20 |train loss: 0.8528 | train acc: 0.6209 |test loss: 1.0842 | test acc: 0.6209
